We need to break out columns into the pre-stop observables - e.g. reason for stop, age / weight / build, precinct, xcoord/ycoord
from the during-stop features

In [81]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score
from imblearn.over_sampling import SMOTE
import data_cleaner as dc
import clean_cat_values
import data_modeler as dm
import importlib
import data_dicts
importlib.reload(data_dicts)
importlib.reload(clean_cat_values)
importlib.reload(dc)
importlib.reload(dm)

<module 'data_modeler' from '/Users/brad/Documents/Scripts/flatiron/stop-and-frisk/data_modeler.py'>

In [ ]:
full_df = dc.load_full_sqf(force=True)

Loading 2003...
Loading 2004...
Loading 2005...


In [10]:
full_df.pct_sector.value_counts()

75-     59296
103-    42183
73-K    35768
73-I    32993
77-     31766
        ...  
121-       33
50-5       27
52-Q        5
9-F         2
73-1        1
Name: pct_sector, Length: 860, dtype: int64

In [16]:
pd.options.display.max_rows=119
pd.options.display.max_colwidth=100
pd.options.display.max_columns=119

In [11]:
df = full_df.sample(5000)

In [12]:
df.describe()

,year,pct,ser_num,perobs,perstop,explnstp,othpers,arstmade,sumissue,compyear,...,state,addrpct,beat,xcoord,ycoord,linecm,detailcm,height,wepfound,detail1_
count,5000.000000,5000.000000,5000.000000,4994.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,4982.0,...,0.0,4164.000000,1452.000000,3.999000e+03,3999.000000,3656.000000,4164.000000,4998.000000,5000.000000,508.000000
mean,2008.477800,67.936200,6757.369000,2.658020,5.436800,0.996200,0.239200,0.05980,0.063200,0.0,...,NaN,67.889769,8.290634,1.003772e+06,204075.647162,0.971554,38.067963,68.562625,0.000200,37.757874
std,2.846107,32.830219,22833.899389,4.895333,4.659436,0.061533,0.426638,0.23714,0.243347,0.0,...,NaN,32.588557,6.415294,3.771573e+04,29316.693703,0.166267,25.703878,4.069142,0.014142,24.815581
min,2003.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,...,NaN,1.000000,1.000000,5.967940e+05,124923.000000,0.000000,6.000000,0.000000,0.000000,9.000000
25%,2006.000000,41.000000,1769.000000,1.000000,3.000000,1.000000,0.000000,0.00000,0.000000,0.0,...,NaN,42.000000,4.000000,9.961550e+05,182794.000000,1.000000,20.000000,67.000000,0.000000,20.000000
50%,2009.000000,73.000000,4401.000000,1.000000,5.000000,1.000000,0.000000,0.00000,0.000000,0.0,...,NaN,73.000000,8.000000,1.005166e+06,197008.000000,1.000000,28.000000,69.000000,0.000000,30.500000
75%,2011.000000,101.000000,8819.250000,3.000000,5.000000,1.000000,0.000000,0.00000,0.000000,0.0,...,NaN,101.000000,11.000000,1.016462e+06,232339.000000,1.000000,46.000000,71.000000,0.000000,46.000000
max,2018.000000,123.000000,903368.000000,120.000000,100.000000,1.000000,1.000000,1.00000,1.000000,0.0,...,NaN,123.000000,62.000000,1.065109e+06,268990.000000,1.000000,113.000000,91.000000,1.000000,112.000000


In [34]:
df.crimsusp.value_counts().head(40)

CPW                               748
FEL                               526
ROBBERY                           459
MISD                              395
FELONY                            386
GLA                               279
BURGLARY                          269
CRIMINAL TRESPASS                 155
CRIM TRES                         129
ASSAULT                            98
GRAND LARCENY                      88
CPCS                               88
CPM                                87
BURG                               67
CSCS                               63
GRAND LARCENY AUTO                 56
PETIT LARCENY                      36
MIS                                30
MISDEMEANOR                        30
ROB                                29
CRIM TRESPASS                      25
CRIM TRESS                         24
DRUG SALES                         21
CRIMINAL SALE OF CONTROLLED SU     21
CRIMINAL TRESSPASS                 19
FELONY/ROBBERY                     18
F           

# Modeling on the yes-no columns

In [182]:
y = df.arstmade

In [183]:
X_y_n = df.select_dtypes(include=['int8']).drop(columns='arstmade')

In [184]:
split = dm.load_split(X_y_n, y)

## Balance classes

In [185]:
split['y_train'].value_counts(normalize=True)

0    0.934933
1    0.065067
Name: arstmade, dtype: float64

In [186]:
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_sample(split['X_train'], split['y_train']) 
y_train_resampled.value_counts()

1    3506
0    3506
Name: arstmade, dtype: int64

In [187]:
logit = LogisticRegression(solver='saga', penalty='l1', max_iter=4000)

In [188]:
logit.fit(X_train_resampled, y_train_resampled)
y_hat_train = logit.predict(split['X_train'])
residuals = np.abs(split['y_train'] - y_hat_train)
print(pd.Series(residuals).value_counts())
print(pd.Series(residuals).value_counts(normalize=True))

0    3370
1     380
Name: arstmade, dtype: int64
0    0.898667
1    0.101333
Name: arstmade, dtype: float64


In [189]:
pd.Series(y_hat_train).value_counts()

0    3248
1     502
dtype: int64

In [190]:
y_hat_test = logit.predict(split['X_test'])
residuals = np.abs(split['y_test'] - y_hat_test)
print(pd.Series(residuals).value_counts())
print(pd.Series(residuals).value_counts(normalize=True))

0    1119
1     131
Name: arstmade, dtype: int64
0    0.8952
1    0.1048
Name: arstmade, dtype: float64


In [191]:
logit.coef_

array([[ 1.15422117, -0.63778506, -2.76921075, -0.43906889,  1.53664947,
         4.71244694,  5.92921729, -2.63908453,  0.        ,  0.        ,
         0.        , -0.02803667,  0.        ,  0.        , -1.20198606,
        -0.6356687 , -2.01897471, -1.57564569, -0.33389638,  0.21683498,
         2.36879078,  0.        , -2.45493293, -1.06587496, -0.05039847,
        -2.07659287,  0.33961796, -1.04238158, -0.96452628, -0.92649511,
        -3.13833436, -0.80436542, -2.71756743, -2.22544411, -0.58363806,
        -0.37144361, -1.7388306 ,  0.        , -2.33176135, -2.07317647,
        -0.25440522, -0.63755845,  0.        , -2.14312206, -2.25814912,
        -1.29766975, -0.73734479, -0.16672737, -1.79909294, -0.64480502,
        -2.15552186, -4.068843  , -1.81998161,  2.25932235,  0.53537669,
        -0.57391598, -2.2129033 ,  0.        ]])

In [192]:
split['X_train'].columns

Index(['explnstp', 'othpers', 'sumissue', 'offunif', 'frisked', 'searched',
       'contrabn', 'adtlrept', 'pistol', 'riflshot', 'asltweap', 'knifcuti',
       'machgun', 'othrweap', 'pf_hands', 'pf_wall', 'pf_grnd', 'pf_drwep',
       'pf_ptwep', 'pf_baton', 'pf_hcuff', 'pf_pepsp', 'pf_other', 'radio',
       'ac_rept', 'ac_inves', 'rf_vcrim', 'rf_othsw', 'ac_proxm', 'rf_attir',
       'cs_objcs', 'cs_descr', 'cs_casng', 'cs_lkout', 'rf_vcact', 'cs_cloth',
       'cs_drgtr', 'ac_evasv', 'ac_assoc', 'cs_furtv', 'rf_rfcmp', 'ac_cgdir',
       'rf_verbl', 'cs_vcrim', 'cs_bulge', 'cs_other', 'ac_incid', 'ac_time',
       'rf_knowl', 'ac_stsnd', 'ac_other', 'sb_hdobj', 'sb_outln', 'sb_admis',
       'sb_other', 'rf_furt', 'rf_bulg', 'wepfound'],
      dtype='object')

In [193]:
coef = pd.DataFrame(logit.coef_)
coef.columns = split['X_train'].columns
coef.T

,0
explnstp,1.154221
othpers,-0.637785
sumissue,-2.769211
offunif,-0.439069
frisked,1.536649
searched,4.712447
contrabn,5.929217
adtlrept,-2.639085
pistol,0.000000
riflshot,0.000000


In [60]:
df.groupby(by=['sumissue','arstmade']).count().iloc[:,0]

sumissue  arstmade
0         0           4372
          1            284
1         0            332
          1             12
Name: year, dtype: int64

## Only pre-stop observables

In [205]:
X_pre_stop = df[list(set(dc.Y_N_COLS).intersection(dm.PRE_STOP_OBSERVABLES))]

In [221]:
split, logit = dm.split_and_logit(X_y_n, y)

Train
 0    3370
1     380
Name: arstmade, dtype: int64
0    0.898667
1    0.101333
Name: arstmade, dtype: float64
Test
 0    1117
1     133
Name: arstmade, dtype: int64
0    0.8936
1    0.1064
Name: arstmade, dtype: float64


In [206]:
split, logit = dm.split_and_logit(X_pre_stop, y)

Train
 0    2503
1    1247
Name: arstmade, dtype: int64
0    0.667467
1    0.332533
Name: arstmade, dtype: float64
Test
 0    824
1    426
Name: arstmade, dtype: int64
0    0.6592
1    0.3408
Name: arstmade, dtype: float64


In [219]:
balanced_accuracy_score(split['y_test'], np.zeros(split['y_test'].shape))

0.5

In [222]:
balanced_accuracy_score(split['y_test'], logit.predict(split['X_test']))

0.7847765715728352

In [225]:
df.build.value_counts()

M    2947
T    1530
H     430
Z      74
U      19
Name: build, dtype: int64

In [227]:
df.race.value_counts(normalize=True)

B                                 0.5180
Q                                 0.2442
W                                 0.0906
P                                 0.0650
Z                                 0.0370
A                                 0.0324
U                                 0.0044
I                                 0.0036
BLACK                             0.0028
WHITE HISPANIC                    0.0012
WHITE                             0.0004
BLACK HISPANIC                    0.0004
X                                 0.0000
ASIAN/PAC.ISL                     0.0000
ASIAN / PACIFIC ISLANDER          0.0000
AMERICAN INDIAN/ALASKAN NATIVE    0.0000
AMER IND                          0.0000
Name: race, dtype: float64

In [228]:
full_df.race.value_counts(normalize=True)

B                                 0.515874
Q                                 0.242384
W                                 0.098652
P                                 0.061078
Z                                 0.038161
A                                 0.030701
U                                 0.004500
I                                 0.003928
BLACK                             0.002473
WHITE HISPANIC                    0.000960
X                                 0.000420
WHITE                             0.000402
BLACK HISPANIC                    0.000379
ASIAN / PACIFIC ISLANDER          0.000043
ASIAN/PAC.ISL                     0.000040
AMERICAN INDIAN/ALASKAN NATIVE    0.000003
AMER IND                          0.000002
Name: race, dtype: float64

In [229]:
full_df[full_df.year==2017].race.value_counts(normalize=True)

BLACK                             0.579202
WHITE HISPANIC                    0.226084
WHITE                             0.088623
BLACK HISPANIC                    0.086702
ASIAN/PAC.ISL                     0.018566
AMER IND                          0.000823
Z                                 0.000000
AMERICAN INDIAN/ALASKAN NATIVE    0.000000
ASIAN / PACIFIC ISLANDER          0.000000
B                                 0.000000
I                                 0.000000
X                                 0.000000
P                                 0.000000
Q                                 0.000000
U                                 0.000000
W                                 0.000000
A                                 0.000000
Name: race, dtype: float64

In [231]:
df17 = dc.load_sqf(2017, convert=False)

Loading 2017...


In [236]:
df17.SUSPECT_EYE_COLOR.value_counts(normalize=True)

BRO    0.850370
BLK    0.089673
ZZZ    0.022484
BLU    0.015599
HAZ    0.009237
GRN    0.008976
GRY    0.001394
BR     0.001133
OTH    0.000349
MED    0.000261
THN    0.000174
HEA    0.000174
MUL    0.000087
MAR    0.000087
Name: SUSPECT_EYE_COLOR, dtype: float64

In [461]:
x = pd.Series(['A', np.NaN, 'B'])
x.fillna('').asty

0    A
1     
2    B
dtype: object

In [459]:
full_df.pct.astype(int)

0           70
1           70
2           68
3           48
4           48
          ... 
5087345    121
5087346     32
5087347     32
5087348     32
5087349     79
Name: pct, Length: 5087350, dtype: int64

In [238]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 3079204 to 1353209
Columns: 118 entries, year to detail1_
dtypes: Int64(7), category(20), datetime64[ns](1), float64(14), int8(59), object(17)
memory usage: 1.9+ MB


In [239]:
df.build.value_counts()

M    2947
T    1530
H     430
Z      74
U      19
Name: build, dtype: int64

In [240]:
ONE_HOT_ENCODE = ['build', 'pct']

In [354]:
cats = df.select_dtypes('category').columns.to_list()
cats

['recstat',
 'inout',
 'trhsloc',
 'typeofid',
 'officrid',
 'offverb',
 'offshld',
 'sex',
 'race',
 'haircolr',
 'eyecolor',
 'build',
 'addrtyp',
 'city',
 'sector',
 'dettypcm',
 'month',
 'day',
 'forceuse']

In [355]:
df[cats].describe()

,recstat,inout,trhsloc,typeofid,officrid,offverb,offshld,sex,race,haircolr,eyecolor,build,addrtyp,city,sector,dettypcm,month,day,forceuse
count,4857,4967,3191,4954,5000.0,5000.0,5000.0,5000,4999,5000,5000,5000,4967,4248,4109,4247,5000,5000,121
unique,3,3,3,4,2.0,2.0,2.0,3,8,10,11,5,2,5,18,2,16,16,6
top,A,O,P,P,0.0,0.0,0.0,M,B,BK,BR,M,L,BROOKLYN,A,CM,<bound method PandasDelegate._add_delegate_accessors.<locals>._create_delegator_method.<locals>....,<bound method PandasDelegate._add_delegate_accessors.<locals>._create_delegator_method.<locals>....,DS
freq,3498,3855,2115,2692,4988.0,4049.0,3612.0,4538,2526,3712,4381,2947,3545,1537,460,4235,722,722,56


In [352]:
df.sumissue.value_counts().sum()

4972

In [416]:
sectors = pd.DataFrame(full_df[['sector','pct']].dropna().groupby(['sector','pct']).size().reset_index().rename(columns={0:'ct'}))

In [418]:
sectors.query('sector=="A"').groupby()

,sector,pct,ct
2,A,1,1626
3,A,5,4109
4,A,6,2182
5,A,7,7769
6,A,9,12694
7,A,10,5682
8,A,13,1623
9,A,14,1637
10,A,17,1682
11,A,18,499


In [419]:
sectors.sector.value_counts()

B    77
A    77
C    77
D    77
E    75
F    72
G    69
H    60
I    52
J    43
K    33
M    24
N    15
P    12
Q     7
R     6
T     3
U     2
5     1
1     1
Name: sector, dtype: int64

In [404]:
pd.DataFrame(sp.groupby(['sector','pct']).size())

0
sector pct       
1      73       1
5      50      27
A      1     1626
       5     4109
       6     2182
...           ...
T      44    6767
       109   1163
       120  13952
U      44    2394
       109    757

[783 rows x 1 columns]

In [395]:
full_df.columns

Index(['year', 'pct', 'ser_num', 'datestop', 'timestop', 'recstat', 'inout',
       'trhsloc', 'perobs', 'crimsusp',
       ...
       'dettypcm', 'linecm', 'detailcm', 'datetimestop', 'month', 'day',
       'height', 'wepfound', 'forceuse', 'detail1_'],
      dtype='object', length=118)

In [363]:
len(full_df)

5085961

In [314]:
df[~df.forceuse.isna()].year.value_counts()

2012    66
2013    28
2014     6
2015     4
2016     1
2018     0
2017     0
2011     0
2010     0
2009     0
2008     0
2007     0
2006     0
2005     0
2004     0
2003     0
Name: year, dtype: int64

In [315]:
full_df.forceuse.value_counts()

DS    49859
SF    25284
OT    24945
SW     8585
OR     4048
DO     1744
Name: forceuse, dtype: int64

In [351]:
df = dc.clean_categories(df)

In [357]:
full_df.race.value_counts()

B    2636035
Q    1237520
W     503734
P     312534
Z     194067
A     156554
U      22883
I      20003
X       2137
Name: race, dtype: int64

In [42]:
list(set(dc.REPLACE_REVERSE_DICT['premname']) | set(dc.USE_OTHER_VALUES['premname']))

['HALLWAY',
 'SUBWAY',
 'COMMERCIAL',
 'WALKWAY',
 'BUILDING',
 'RESIDENTIAL',
 'SIDEWALK',
 'PARK',
 'STREET',
 'MEZZ',
 'STAIRWELL',
 'LOBBY',
 'PLAYGROUND',
 'PARKING LOT',
 'NYCHA',
 'CAR']

In [33]:
{ 'premname' : { val : key for key, val_list in dc.PREMNAME_REPLACE_DICT.items() for val in val_list}}

{'premname': {'ST': 'STREET',
  'PUBLIC STREET': 'STREET',
  'STREET CORNER': 'STREET',
  'MEZZANINE': 'MEZZ',
  'PUBLIC SIDEWALK': 'SIDEWALK',
  'APT BUILDING': 'RESIDENTIAL',
  'APARTMENT': 'RESIDENTIAL',
  'APT': 'RESIDENTIAL',
  'RES': 'RESIDENTIAL',
  'APT. BUILDING': 'RESIDENTIAL',
  'HOUSE': 'RESIDENTIAL',
  'PRIVATE HOUSE': 'RESIDENTIAL',
  'RESIDENCE': 'RESIDENTIAL',
  'APARTMENT BUILDING': 'RESIDENTIAL',
  'BLDG': 'BUILDING',
  'STAIRCASE': 'STAIRWELL',
  'COMM': 'COMMERCIAL',
  'STORE': 'COMMERCIAL'}}

In [46]:
df.age.value_counts()

20.0     309
18.0     293
19.0     291
17.0     255
22.0     240
21.0     236
16.0     208
23.0     201
24.0     191
25.0     178
15.0     155
26.0     153
27.0     125
28.0     119
29.0     118
30.0     112
31.0     111
35.0      88
37.0      84
32.0      83
40.0      80
34.0      79
36.0      76
38.0      74
33.0      74
14.0      71
39.0      69
45.0      67
42.0      64
43.0      60
46.0      59
44.0      57
41.0      56
50.0      48
48.0      47
0.0       47
47.0      44
49.0      36
51.0      31
52.0      29
55.0      25
13.0      25
54.0      23
53.0      21
56.0      19
99.0      18
58.0      18
57.0      17
61.0      13
60.0      12
59.0      11
63.0      10
1.0        9
64.0       7
66.0       7
62.0       6
71.0       3
12.0       3
69.0       3
999.0      3
68.0       3
65.0       2
74.0       2
10.0       2
72.0       2
67.0       2
145.0      1
70.0       1
80.0       1
263.0      1
81.0       1
5.0        1
73.0       1
11.0       1
305.0      1
203.0      1
75.0       1

In [79]:
df04 = None
df04 = dc.load_sqf(2004)

Loading 2004...


/Users/brad/Documents/Scripts/flatiron/stop-and-frisk/data_cleaner.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['height'] = data['ht_feet'] * 12 + data['ht_inch']
/Users/brad/Documents/Scripts/flatiron/stop-and-frisk/data_cleaner.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['pct_sector'] = data.pct.astype(int).astype(str) + '-' + data.sector.astype('object').fillna('').astype(str)
/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A

{'STREET': ['ST', 'PUBLIC STREET', 'STREET CORNER'], 'MEZZ': ['MEZZANINE'], 'SIDEWALK': ['PUBLIC SIDEWALK'], 'RESIDENTIAL': ['APT BUILDING', 'APARTMENT', 'APT', 'RES', 'APT. BUILDING', 'HOUSE', 'PRIVATE HOUSE', 'RESIDENCE', 'APARTMENT BUILDING'], 'BUILDING': ['BLDG'], 'STAIRWELL': ['STAIRCASE'], 'COMMERCIAL': ['COMM', 'STORE']}
{'FELONY': ['FEL', 'F'], 'ASSAULT': ['ASSAULT 3'], 'MISDEMEANOR': ['M', 'MISD', 'MIS/CPW'], 'CRIMINAL POSSESSION WEAPON': ['CPW', 'CPW 3', 'CPW 4', 'C.P.W.', 'CRIMINAL POSSESSION OF WEAPON', 'C.P.W', 'CPW GUN', 'CPW FIREARM', 'FELONY CPW'], 'CRIMINAL TRESPASS': ['CRIMINAL TRESSPASS', 'CRIM TRES', 'CRIM TRESS', 'TRESPASSING', 'CRIMINAL  TRESSPASS', 'CRIM. TRESP.', 'CRIMINAL TRES', 'CRIM TRESSPASS', 'MIS/CRIM TRES'], 'BURGLARY': ['BURG', 'BURG.'], 'ROBBERY': ['ROB', 'ROBBERY PATTERN', 'ROBBERY/CPW', 'CPW/ROBBERY', 'ROBB', 'ROBBERY 1', 'FELONY/ROBBERY'], 'GRAND LARCENY AUTO': ['GLA', 'GLA - GRAND LARCENY AUTO', 'G.L.A.', 'G.L.A', 'GRAND LARCENY FROM AUTO', 'GLA/CPW

In [80]:
df04.crimsusp.value_counts()

CRIMINAL POSSESSION WEAPON                  62188
OTHER                                       59871
CRIMINAL TRESPASS                           41643
BURGLARY                                    32574
GRAND LARCENY AUTO                          32560
ROBBERY                                     29948
CRIMINAL SALE CONTROLLED SUBSTANCE          14555
CRIMINAL POSSESSION CONTROLLED SUBSTANCE    10787
ASSAULT                                      7602
GRAND LARCENY                                2955
PETIT LARCENY                                2330
CRIMINAL SALE MARIJUANA                      1373
PROSTITUTION                                 1109
FELONY                                        721
MISDEMEANOR                                   485
CRIMINAL POSSESSION MARIJUANA                  14
Name: crimsusp, dtype: int64

In [77]:
df04b = dc.load_sqf(2004, engineer=False)

Loading 2004...


In [78]:
df04b.crimsusp.value_counts().head(100)

CPW                               62593
CRIMINAL TRESPASS                 34863
BURGLARY                          32048
ROBBERY                           30114
GRAND LARCENY AUTO                25643
CRIMINAL POSSESSION OF CONTROL     7443
CRIMINAL SALE OF CONTROLLED SU     6821
GLA                                6632
ASSAULT                            6408
DRUG SALES                         5925
CRIMINAL POSSESSION OF MARIHUA     5187
CRIMINAL TRESSPASS                 4047
CRIMINAL MISCHIEF                  3295
GRAND LARCENY                      3139
CPSP                               2701
CPCS                               2500
PETIT LARCENY                      2438
TRESPASS                           2386
BURG                               1678
CSCS                               1660
CRIM TRESPASS                      1589
CRIM TRES                          1567
ASSAULT 3                          1544
GLA - GRAND LARCENY AUTO           1366
CPM                                1267
